In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

построить распределния переменных
проверить на нормальность?
Возраст - категориальная переменная

попробовтаь линейнуюрегрессию, бинарное дерево
Линейная регрессия: кол-во детей сделать как кат переменную : 0, 1, 2, более 2

Let's download the data and see the first rows.

In [ ]:
data = pd.read_csv('../input/insurance/insurance.csv')
data.head()

In [ ]:
data.shape

The data contains 1 338 rows and 7 columns. 

In [ ]:
data.info()

We have 7 columns: age, sex, bmi, children, smoker, region, charges.

In [ ]:
data.describe()

According to the table we see next:
    - Age is between 18 and 64 years. The mean age is 39.
    - BMI is between 15.96 and 53.13. The mean bmi is 30.66.
    - The numbers of children are between 0 and 5. The mean number of children is 1.09.
    - Charges are between 1121.87 and 63770.43. The mean charges are 13 270.42.
    
Let's see the distribution of these features.

In [ ]:
data.sort_values('charges', ascending = False).head(10)

In [ ]:
#charges = data["charges"]
sns.distplot(data.age, bins = 30, color = '#50248f')
# plt.title('Распределение целевой переменной - y\n',fontsize=15)
# plt.xlabel('Значение в секундах'); plt.ylabel('Кол-во (частота)')
plt.show()

The distribution of age is not normal. There are too many people at 20 age (200+people).

In [ ]:
ranges = [17,25,35,45,55,64]
labels = ['17-25', '25-35', '35-45', '45-55', '55-64']
data['agebins'] = pd.cut(data.age, ranges, labels = labels, ordered = True)
data.head(15)

In [ ]:
data.agebins.value_counts()

In [ ]:
sns.distplot(data[data.agebins == '17-25'].age)
plt.show()

In [ ]:
print(data[data.age == 18].charges.mean())
print(data[data.age == 19].charges.mean())


In [ ]:
sns.distplot(data.bmi)
plt.show()

The distribution of bmi looks like normal distribution.

In [ ]:
sns.distplot(data.children, kde = False)
plt.show()

In [ ]:
data.children.value_counts()

As we can see the most of the surveyed people doesn't have children (574 persons). 
324 persons have 1 child
240 persons have 2 children 
157 persons have 3 children
25 persons have 4 children
18 persons have 5 children

In [ ]:
sns.distplot(data.charges)
plt.show()

In [ ]:
data.isna().sum()

The dataset does not contain empty values

In [ ]:
data['sex_bin'] = data.apply(lambda row:0 if row['sex'] == 'male' else 1, axis = 1)
data.head()

In [ ]:
data['smoker_bin'] = data.apply(lambda row:0 if row['smoker'] == 'yes' else 1, axis = 1)
data.head()

In [ ]:
data.groupby('sex').charges.mean()

We see that individual medical costs billed by health insurance are higher for male (13 956.75) than for female (12 569.58).

In [ ]:
data.groupby('smoker').charges.mean()

We see that individual medical costs billed by health insurance are higher for smoker (32 050.23) than for non-smoker (8 434.27).

In [ ]:
data.groupby(['sex', 'smoker']).charges.mean()

May be smoking are more affected on men's medical costs than on women's.

In [ ]:
data.groupby('region').charges.mean()

We see that the highest individual medical costs are billed in southeast region, and the smallest are in southwest region.

In [ ]:
# Plot a histogram that shows the distribution of charges in the dataset:

charges = data["charges"]
sns.distplot(charges)
plt.show()

In [ ]:
# Plot a violinplot, which shows the distribution of charges by different sex:

sns.violinplot(data=data, x="sex", y="charges")
plt.show()

In [ ]:
# Plot a violinplot, which shows the distribution of charges for smoker and non-smoker:

sns.violinplot(data=data, x="smoker", y="charges")
plt.show()

In [ ]:
# Plot a violinplot, which shows the distribution of charges in different locations:

sns.violinplot(data=data, x="region", y="charges")
plt.show()

## Linear Regression

In [ ]:
data_reg = data[['age', 'bmi', 'children', 'charges', 'sex_bin', 'smoker_bin']]
data_reg.head()

In [ ]:
pair_grid_plot = sns.PairGrid(data_reg)
pair_grid_plot.map(plt.scatter)

In [ ]:
data_reg.corr()

In [ ]:
data_reg[data_reg.charges < 20000].corr()